In [4]:
using Distributions
using Random

In [20]:
beta = [1, 2, 0,0,0,0,0,0,0]

9

In [33]:
# sample data creation
Random.seed!(123)
du = Uniform()
Xvar = rand(du, 50, 8)
constant = ones(50)
X = hcat(constant, Xvar*2)

dn = Normal()
eps = rand(dn, 50)

y = X*beta + eps;